# Predict heart failure with Watson Machine Learning

![](https://www.cdc.gov/vitalsigns/heartdisease-stroke/images/graph4_980px.jpg)

This notebook contains steps and code to create a predictive model to predict heart failure and then deploy that model to Watson Machine Learning so it can be used in an application.

## Learning Goals

The learning goals of this notebook are:

* Load a CSV file into the Object Storage service linked to your Watson Studio
* Create an Apache Spark machine learning model
* Train and evaluate a model
* Persist a model in a Watson Machine Learning repository

## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

* Create a Watson Machine Learning service instance (a free plan is offered) and associate it with your project
* Upload heart failure data to the Object Store service that is part of Watson Studio

We'll be using a few libraries for this exercise:

1. [Watson Machine Learning Client](http://wml-api-pyclient.mybluemix.net/): Client library to work with the Watson Machine Learning service on IBM Cloud.
1. [Pixiedust](https://github.com/pixiedust/pixiedust): Python Helper library for Jupyter Notebooks
1. [ibmos2spark](https://github.com/ibm-watson-data-lab/ibmos2spark): Facilitates Data I/O between Spark and IBM Object Storage services

In [1]:
!pip install --upgrade ibmos2spark
!pip install --upgrade pixiedust
!pip install --upgrade watson-machine-learning-client

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200404012500-0000
KERNEL_ID = 7af4e862-141e-421b-a694-8c50486b14c6
    100% |################################| 204kB 2.6MB/s eta 0:00:01
    100% |################################| 798kB 2.9MB/s eta 0:00:01
    100% |################################| 5.8MB 1.6MB/s eta 0:00:01
    100% |################################| 92kB 3.7MB/s eta 0:00:01
    100% |################################| 61kB 2.2MB/s eta 0:00:01
    100% |################################| 583kB 3.5MB/s eta 0:00:01
    100% |################################| 143kB 3.8MB/s eta 0:00:01
    100% |################################| 133kB 4.2MB/s eta 0:00:01
    100% |################################| 61kB 2.3MB/s eta 0:00:01
    100% |################################| 163kB 3.9MB/s eta 0:00:01
  Stored in directory: /home/spark/shared/.cache/pip/wheels/e8/b1/86/c2f2e16e6bf9bfe556f9dbf8adb9f41816c476d73078c7d0eb
  Stored in directory: /hom

    100% |################################| 512kB 3.6MB/s eta 0:00:01
    100% |################################| 235kB 3.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/65/eb/1f97cb97bfc2390a276969c6fae16075da282f5058082d4cb10c6c5c1dba/six-1.14.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/89/e3/afebe61c546d18fb1709a61bee788254b40e736cff7271c7de5de2dc4128/idna-2.9-py2.py3-none-any.whl
    100% |################################| 768kB 3.7MB/s eta 0:00:01
    100% |################################| 225kB 3.9MB/s eta 0:00:01
    100% |################################| 552kB 3.5MB/s eta 0:00:01
  Stored in directory: /home/spark/shared/.cache/pip/wheels/37/9c/c4/a2c610ccb877d37c2cb87a5bfe55845fecffd6bb01bcd5e9d5
  Stored in directory: /home/spark/shared/.cache/pip/wheels/75/93/e

## 2. Load and explore data

In this section you will load the data as an Apache Spark DataFrame and perform a basic exploration. Load the data to the Spark DataFrame from your associated Object Storage instance.

> **IMPORTANT**: Follow the lab instructions to insert an Apache Spark DataFrame in the cell below.

> **IMPORTANT**: Ensure the DataFrame is named `df_data`.

> **IMPORTANT**: Add `.option('inferSchema','True')\` to the inserted code.

In [3]:

import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-67e1c394-e358-4d3d-b74d-9eadcb1d622b',
    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'api_key': 'LUcMy6n3mwZ6kvGzH3ZeTJjmQXzygFJ29RD5fsDZZJoF'
}

configuration_name = 'os_1cd9e0470669462fa5b1f70fe9aa0a9e_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_1 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema','True')\
  .load(cos.url('patientdataV6.csv', 'heartfailureratepredictor-donotdelete-pr-i0llodm3ejqxwp'))
df_data_1.take(5)
  

[Row(AVGHEARTBEATSPERMIN=93, PALPITATIONSPERDAY=22, CHOLESTEROL=163, BMI=25, HEARTFAILURE='N', AGE=49, SEX='F', FAMILYHISTORY='N', SMOKERLAST5YRS='N', EXERCISEMINPERWEEK=110),
 Row(AVGHEARTBEATSPERMIN=108, PALPITATIONSPERDAY=22, CHOLESTEROL=181, BMI=24, HEARTFAILURE='N', AGE=32, SEX='F', FAMILYHISTORY='N', SMOKERLAST5YRS='N', EXERCISEMINPERWEEK=192),
 Row(AVGHEARTBEATSPERMIN=86, PALPITATIONSPERDAY=0, CHOLESTEROL=239, BMI=20, HEARTFAILURE='N', AGE=60, SEX='F', FAMILYHISTORY='N', SMOKERLAST5YRS='N', EXERCISEMINPERWEEK=121),
 Row(AVGHEARTBEATSPERMIN=80, PALPITATIONSPERDAY=36, CHOLESTEROL=164, BMI=31, HEARTFAILURE='Y', AGE=45, SEX='F', FAMILYHISTORY='Y', SMOKERLAST5YRS='N', EXERCISEMINPERWEEK=141),
 Row(AVGHEARTBEATSPERMIN=66, PALPITATIONSPERDAY=36, CHOLESTEROL=185, BMI=23, HEARTFAILURE='N', AGE=39, SEX='F', FAMILYHISTORY='N', SMOKERLAST5YRS='N', EXERCISEMINPERWEEK=63)]

Explore the loaded data by using the following Apache® Spark DataFrame methods:

* `df_data.printSchema` to print the data schema
* `df_data.describe()` to print the top twenty records
* `df_data.count()` to count all records

In [5]:
df_data_1.printSchema()

root
 |-- AVGHEARTBEATSPERMIN: integer (nullable = true)
 |-- PALPITATIONSPERDAY: integer (nullable = true)
 |-- CHOLESTEROL: integer (nullable = true)
 |-- BMI: integer (nullable = true)
 |-- HEARTFAILURE: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- SEX: string (nullable = true)
 |-- FAMILYHISTORY: string (nullable = true)
 |-- SMOKERLAST5YRS: string (nullable = true)
 |-- EXERCISEMINPERWEEK: integer (nullable = true)



As you can see, the data contains ten  fields. The  HEARTFAILURE field is the one we would like to predict (label).

In [6]:
df_data_1.show()

+-------------------+------------------+-----------+---+------------+---+---+-------------+--------------+------------------+
|AVGHEARTBEATSPERMIN|PALPITATIONSPERDAY|CHOLESTEROL|BMI|HEARTFAILURE|AGE|SEX|FAMILYHISTORY|SMOKERLAST5YRS|EXERCISEMINPERWEEK|
+-------------------+------------------+-----------+---+------------+---+---+-------------+--------------+------------------+
|                 93|                22|        163| 25|           N| 49|  F|            N|             N|               110|
|                108|                22|        181| 24|           N| 32|  F|            N|             N|               192|
|                 86|                 0|        239| 20|           N| 60|  F|            N|             N|               121|
|                 80|                36|        164| 31|           Y| 45|  F|            Y|             N|               141|
|                 66|                36|        185| 23|           N| 39|  F|            N|             N|            

In [7]:
df_data_1.describe().show()

+-------+-------------------+------------------+------------------+------------------+------------+------------------+-----+-------------+--------------+------------------+
|summary|AVGHEARTBEATSPERMIN|PALPITATIONSPERDAY|       CHOLESTEROL|               BMI|HEARTFAILURE|               AGE|  SEX|FAMILYHISTORY|SMOKERLAST5YRS|EXERCISEMINPERWEEK|
+-------+-------------------+------------------+------------------+------------------+------------+------------------+-----+-------------+--------------+------------------+
|  count|              10800|             10800|             10800|             10800|       10800|             10800|10800|        10800|         10800|             10800|
|   mean|  87.11509259259259|20.423148148148147|195.08027777777778| 26.35972222222222|        null|49.965185185185184| null|         null|          null|119.72953703703703|
| stddev| 19.744375148984474|12.165320351622993|26.136731865042325|3.8201472810942136|        null|13.079280962015586| null|         nu

In [8]:
df_data_1.count()

10800

As you can see, the data set contains 10800 records.

## 3. Interactive Visualizations w/PixieDust

In [9]:
import pixiedust

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table SPARK_PACKAGES created successfully
Table USER_PREFERENCES created successfully
Table service_connections created successfully


### Simple visualization using bar charts

With PixieDust's `display()` method you can visually explore the loaded data using built-in charts, such as, bar charts, line charts, scatter plots, or maps.
To explore a data set: choose the desired chart type from the drop down, configure chart options, configure display options.

In [11]:
display(df_data_1)

AVGHEARTBEATSPERMIN,PALPITATIONSPERDAY,CHOLESTEROL,BMI,HEARTFAILURE,AGE,SEX,FAMILYHISTORY,SMOKERLAST5YRS,EXERCISEMINPERWEEK
66,36,185,23,N,39,F,N,N,63
92,28,174,22,N,44,F,N,N,107
60,1,194,28,N,71,M,N,N,27
96,29,229,21,N,62,M,Y,N,135
80,19,221,25,N,53,M,N,N,76
79,21,150,30,Y,71,M,Y,N,191
64,37,189,22,N,57,F,N,N,51
103,22,185,26,N,32,F,N,N,113
70,8,223,30,N,54,M,N,N,178
98,28,213,25,N,45,F,N,N,124


## 4. Create a Spark machine learning model

In this section you will learn how to prepare data, create and train a Spark machine learning model.

### 4.1 Prepare data

In this subsection you will split your data into: train and test data sets.

In [12]:
split_data = df_data_1.randomSplit([0.8, 0.20], 24)
train_data = split_data[0]
test_data = split_data[1]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))

Number of training records: 8637
Number of testing records : 2163


As you can see our data has been successfully split into two data sets:

* The train data set, which is the largest group, is used for training.
* The test data set will be used for model evaluation and is used to test the assumptions of the model.

### 4.2 Create pipeline and train a model

In this section you will create a Spark machine learning pipeline and then train the model. In the first step you need to import the Spark machine learning packages that will be needed in the subsequent steps. A sequence of data processing is called a _data pipeline_. Each step in the pipeline processes the data and passes the result to the next step in the pipeline, this allows you to transform and fit your model with the raw input data.

In [13]:
from pyspark.ml.feature import StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the string fields to numeric ones by using the StringIndexer transformer.

In [15]:
stringIndexer_label = StringIndexer(inputCol="HEARTFAILURE", outputCol="label").fit(df_data_1)
stringIndexer_sex = StringIndexer(inputCol="SEX", outputCol="SEX_IX")
stringIndexer_famhist = StringIndexer(inputCol="FAMILYHISTORY", outputCol="FAMILYHISTORY_IX")
stringIndexer_smoker = StringIndexer(inputCol="SMOKERLAST5YRS", outputCol="SMOKERLAST5YRS_IX")


In the following step, create a feature vector by combining all features together.

In [16]:
vectorAssembler_features = VectorAssembler(inputCols=["AVGHEARTBEATSPERMIN","PALPITATIONSPERDAY","CHOLESTEROL","BMI","AGE","SEX_IX","FAMILYHISTORY_IX","SMOKERLAST5YRS_IX","EXERCISEMINPERWEEK"], outputCol="features")

Next, define estimators you want to use for classification. Random Forest is used in the following example.

In [20]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, indexed labels back to original labels.

In [21]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

In [22]:
transform_df_pipeline = Pipeline(stages=[stringIndexer_label, stringIndexer_sex, stringIndexer_famhist, stringIndexer_smoker, vectorAssembler_features])
transformed_df = transform_df_pipeline.fit(df_data_1).transform(df_data_1)
transformed_df.show()

+-------------------+------------------+-----------+---+------------+---+---+-------------+--------------+------------------+-----+------+----------------+-----------------+--------------------+
|AVGHEARTBEATSPERMIN|PALPITATIONSPERDAY|CHOLESTEROL|BMI|HEARTFAILURE|AGE|SEX|FAMILYHISTORY|SMOKERLAST5YRS|EXERCISEMINPERWEEK|label|SEX_IX|FAMILYHISTORY_IX|SMOKERLAST5YRS_IX|            features|
+-------------------+------------------+-----------+---+------------+---+---+-------------+--------------+------------------+-----+------+----------------+-----------------+--------------------+
|                 93|                22|        163| 25|           N| 49|  F|            N|             N|               110|  0.0|   1.0|             0.0|              0.0|[93.0,22.0,163.0,...|
|                108|                22|        181| 24|           N| 32|  F|            N|             N|               192|  0.0|   1.0|             0.0|              0.0|[108.0,22.0,181.0...|
|                 86|    

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [23]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_sex, stringIndexer_famhist, stringIndexer_smoker, vectorAssembler_features, rf, labelConverter])

Now, you can train your Random Forest model by using the previously defined **pipeline** and **training data**.

In [24]:
model_rf = pipeline_rf.fit(train_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [27]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)
print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.864078
Test Error = 0.135922


You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

## 5. Persist model

In this section you will learn how to store your pipeline and model in Watson Machine Learning repository by using Python client libraries.
First, you must import client libraries.

> **IMPORTANT**: Update the `wml_credentials` variable below. Copy and paste the entire credential dictionary, which can be found on the _Service Credentials_ tab of the Watson Machine Learning service instance created on the IBM Cloud.

In [28]:
wml_credentials = {
  "apikey": "kz9R3kzTwUVdi5wGUX7dICHIdqTCNBbU--0fSdXpi3dW",
  "iam_apikey_description": "Auto-generated for key f7b551c6-35c0-4410-b6d0-e139b56945ac",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/dc51a66c139e4561b383305f38264eb7::serviceid:ServiceId-34c71fac-df56-4de4-b42d-39752d94bd41",
  "instance_id": "c9dd9bde-421b-4684-833b-da707999da58",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}

print(wml_credentials)

{'apikey': 'kz9R3kzTwUVdi5wGUX7dICHIdqTCNBbU--0fSdXpi3dW', 'iam_apikey_description': 'Auto-generated for key f7b551c6-35c0-4410-b6d0-e139b56945ac', 'iam_apikey_name': 'Service credentials-1', 'iam_role_crn': 'crn:v1:bluemix:public:iam::::serviceRole:Writer', 'iam_serviceid_crn': 'crn:v1:bluemix:public:iam-identity::a/dc51a66c139e4561b383305f38264eb7::serviceid:ServiceId-34c71fac-df56-4de4-b42d-39752d94bd41', 'instance_id': 'c9dd9bde-421b-4684-833b-da707999da58', 'url': 'https://eu-gb.ml.cloud.ibm.com'}


In [29]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
client = WatsonMachineLearningAPIClient(wml_credentials)
print(client.version)

1.0.378


> **TIP**: Update the cell below with your name, email, and name you wish to give to your model.

Create model artifact (abstraction layer).

In [30]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "SAMSON", 
               client.repository.ModelMetaNames.NAME: "Heart Failure Prediction Model"}
published_model = client.repository.store_model(model=model_rf, pipeline=pipeline_rf, meta_props=model_props, training_data=train_data)


## 5.1 Save pipeline and model

In this subsection you will learn how to save pipeline and model artifacts to your Watson Machine Learning instance.

In [31]:
import json
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "metadata": {
    "guid": "163e0826-aa56-4c91-8f81-f9327e21ec72",
    "url": "https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/c9dd9bde-421b-4684-833b-da707999da58/published_models/163e0826-aa56-4c91-8f81-f9327e21ec72",
    "created_at": "2020-04-04T02:06:21.271Z",
    "modified_at": "2020-04-04T02:06:21.335Z"
  },
  "entity": {
    "runtime_environment": "spark-2.3",
    "learning_configuration_url": "https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/c9dd9bde-421b-4684-833b-da707999da58/published_models/163e0826-aa56-4c91-8f81-f9327e21ec72/learning_configuration",
    "author": {
      "name": "SAMSON"
    },
    "name": "Heart Failure Prediction Model",
    "label_col": "HEARTFAILURE",
    "learning_iterations_url": "https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/c9dd9bde-421b-4684-833b-da707999da58/published_models/163e0826-aa56-4c91-8f81-f9327e21ec72/learning_iterations",
    "training_data_schema": {
      "fields": [
        {
          "metadata": {},
          "name": "AVGHEA


## 5.2 Load model to verify that it was saved correctly

You can load your model to make sure that it was saved correctly.

In [32]:
loaded_model = client.repository.load(published_model_uid)
print(loaded_model)

PipelineModel_481eb61de02affd66d95


Call model against test data to verify that it has been loaded correctly. Examine top 3 results

In [33]:
test_predictions = loaded_model.transform(test_data)
test_predictions.select('probability', 'predictedLabel').show(n=3, truncate=False)

+----------------------------------------+--------------+
|probability                             |predictedLabel|
+----------------------------------------+--------------+
|[0.9219680065846495,0.07803199341535046]|N             |
|[0.9116004662231896,0.08839953377681047]|N             |
|[0.918365186843187,0.08163481315681306] |N             |
+----------------------------------------+--------------+
only showing top 3 rows



## <font color=green>Congratulations</font>, you've sucessfully created a predictive model and saved it in the Watson Machine Learning service. 

You can now switch to the Watson Machine Learning console to deploy the model and then test it in application, or continue within the notebook to deploy the model using the APIs.

***

## 6.0 Accessing Watson ML Models and Deployments through API

Instead of jumping from your notebook into a web browser, manage your model and delopment through a set of APIs

### Deploy model to WML Service

In [35]:
created_deployment = client.deployments.create(published_model_uid, name="Heart Failure prediction")



#######################################################################################

Synchronous deployment creation for uid: '163e0826-aa56-4c91-8f81-f9327e21ec72' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='9cd01603-4f6f-41cc-ac01-3a9fe43c912c'
------------------------------------------------------------------------------------------------




### Print the scoring endpoint

In [36]:
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)

print(scoring_endpoint)

https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/c9dd9bde-421b-4684-833b-da707999da58/deployments/9cd01603-4f6f-41cc-ac01-3a9fe43c912c/online


### List model deployments

In [37]:
client.deployments.list()

------------------------------------  ------------------------  ------  --------------  ------------------------  ---------  -------------
GUID                                  NAME                      TYPE    STATE           CREATED                   FRAMEWORK  ARTIFACT TYPE
9cd01603-4f6f-41cc-ac01-3a9fe43c912c  Heart Failure prediction  online  DEPLOY_SUCCESS  2020-04-04T02:16:04.791Z  mllib-2.3  model
2d53ad7c-84f9-4ee8-8485-5310cf90952f  Heart Failure prediction  online  DEPLOY_SUCCESS  2020-04-04T02:15:20.769Z  mllib-2.3  model
------------------------------------  ------------------------  ------  --------------  ------------------------  ---------  -------------


## 6.1 Invoke prediction model deployment


In [38]:
scoring_payload = { "fields":["AVGHEARTBEATSPERMIN","PALPITATIONSPERDAY","CHOLESTEROL","BMI","AGE","SEX","FAMILYHISTORY","SMOKERLAST5YRS","EXERCISEMINPERWEEK"],"values":[[100,85,242,24,44,"F","Y","Y",125]]}

predictions = client.deployments.score(scoring_endpoint, scoring_payload)

print(json.dumps(predictions, indent=2))
print(predictions['values'][0][17])

{
  "fields": [
    "AVGHEARTBEATSPERMIN",
    "PALPITATIONSPERDAY",
    "CHOLESTEROL",
    "BMI",
    "AGE",
    "SEX",
    "FAMILYHISTORY",
    "SMOKERLAST5YRS",
    "EXERCISEMINPERWEEK",
    "label",
    "SEX_IX",
    "FAMILYHISTORY_IX",
    "SMOKERLAST5YRS_IX",
    "features",
    "rawPrediction",
    "probability",
    "prediction",
    "predictedLabel"
  ],
  "values": [
    [
      100,
      85,
      242,
      24,
      44,
      "F",
      "Y",
      "Y",
      125,
      0.0,
      1.0,
      1.0,
      1.0,
      [
        100.0,
        85.0,
        242.0,
        24.0,
        44.0,
        1.0,
        1.0,
        1.0,
        125.0
      ],
      [
        6.481360088490801,
        13.518639911509199
      ],
      [
        0.32406800442454003,
        0.6759319955754599
      ],
      1.0,
      "Y"
    ]
  ]
}
Y


### Narrow down prediction results to just the prediction

In [39]:
print('Is a 44 year old female that smokes with a low BMI at risk of Heart Failure?: {}'.format(client.deployments.score(scoring_endpoint, scoring_payload)
['values'][0][17]))

Is a 44 year old female that smokes with a low BMI at risk of Heart Failure?: Y
